# Fantasy Football Draft Strategy with MCTS

This notebook implements a Monte Carlo Tree Search (MCTS) based draft strategy that uses:
- **VORP-based player valuations** from your draft board
- **Rookie uncertainty modeling** from your ML pipeline
- **Opponent modeling** using ADP with stochastic sampling
- **Long-horizon planning** via MCTS with value function approximation

## Key Components:
1. **State Representation**: Round, roster composition, available players, picks until next turn
2. **Action Space**: Available players (with masking for roster constraints)
3. **Opponent Model**: Plackett-Luce sampling from ADP with position runs
4. **Reward Function**: Roster-aware VORP with risk penalties and positional scarcity
5. **MCTS Planning**: 400-800 simulations per decision with heuristic value function

## Implementation Strategy:
- Start with MCTS + heuristic value (Option C from the plan)
- Use existing VORP data and rookie uncertainty
- Minimal hyperparameter tuning (focus on λ risk penalty and MCTS simulation count)


## 1. Setup and Data Loading


In [7]:
# Install required packages for Colab with GPU acceleration
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --quiet
%pip install pandas numpy matplotlib seaborn scipy numba cupy-cuda11x --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
import os

# GPU acceleration imports
try:
    import torch
    import cupy as cp
    from numba import cuda, jit

    GPU_AVAILABLE = torch.cuda.is_available()
    print(f"🚀 CUDA Available: {GPU_AVAILABLE}")

    if GPU_AVAILABLE:
        device = torch.cuda.current_device()
        gpu_name = torch.cuda.get_device_name(device)
        gpu_memory = torch.cuda.get_device_properties(device).total_memory / 1e9

        print(f"🎯 GPU Device: {gpu_name}")
        print(f"💾 GPU Memory: {gpu_memory:.1f} GB")
        print(f"🔥 CUDA Cores: {torch.cuda.get_device_properties(device).multi_processor_count}")

        # Optimize for T4 GPU
        if "T4" in gpu_name:
            print("⚡ T4 GPU detected - applying optimizations")
            torch.backends.cudnn.benchmark = True
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True

        # Set memory growth to avoid OOM
        torch.cuda.empty_cache()
    else:
        print("📊 No GPU detected - using CPU (will be slower)")

except ImportError:
    GPU_AVAILABLE = False
    print("📊 GPU libraries not available - installing CPU-only versions")
    print("💡 For GPU acceleration, ensure you're using a GPU runtime")

# Set random seeds for reproducibility
np.random.seed(42)
if 'torch' in globals():
    torch.manual_seed(42)
    if GPU_AVAILABLE:
        torch.cuda.manual_seed(42)

print("✅ Setup complete!")
import seaborn as sns
from scipy.stats import rankdata
from typing import Dict, List, Tuple, Optional, Set
from dataclasses import dataclass, field
from collections import defaultdict, Counter
import random
import math
import pickle
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")


🚀 CUDA Available: True
🎯 GPU Device: Tesla T4
💾 GPU Memory: 15.8 GB
🔥 CUDA Cores: 40
⚡ T4 GPU detected - applying optimizations
✅ Setup complete!
✅ All libraries imported successfully!


## Data Loading

In [8]:
# Robust data loading with multiple fallback methods
# =============================================================================

# Initialize global variables
draft_board = None
adp_data = None
metadata = None
rookie_data = None

def load_data_sources():
    """Load data from multiple sources with proper error handling"""

    global draft_board, adp_data, metadata, rookie_data

    # Try Method 1: Pre-packaged ZIP file (recommended)
    if os.path.exists('colab_data.zip'):
        print("📦 Found colab_data.zip - extracting...")
        try:
            import zipfile
            with zipfile.ZipFile('colab_data.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
            print("✅ ZIP file extracted successfully")

            # Load the data using the colab_loader if it exists
            if os.path.exists('colab_loader.py'):
                print("🔄 Running colab_loader.py...")
                exec(open('colab_loader.py').read())
                return True
            else:
                print("⚠️  colab_loader.py not found in ZIP, loading files directly...")

        except Exception as e:
            print(f"❌ ZIP extraction failed: {e}")

    # Try Method 2: Direct file loading (files already present)
    if os.path.exists('draft_board.csv') and os.path.exists('FantasyPros_2025_Overall_ADP_Rankings.csv'):
        print("📊 Loading data files directly...")
        try:
            draft_board = pd.read_csv('draft_board.csv')
            adp_data = pd.read_csv('FantasyPros_2025_Overall_ADP_Rankings.csv')

            # Load metadata if available
            if os.path.exists('metadata.json'):
                import json
                with open('metadata.json', 'r') as f:
                    metadata = json.load(f)
                print("📋 Loaded configuration metadata")
            else:
                metadata = {'league_settings': {'teams': 12}, 'model_parameters': {}}

            # Load rookie data if available
            if os.path.exists('rookie_data_clean.csv'):
                rookie_data = pd.read_csv('rookie_data_clean.csv')
                print(f"🏈 Loaded rookie data: {len(rookie_data)} players")

            print(f"✅ Loaded {len(draft_board)} players from draft board")
            print(f"✅ Loaded {len(adp_data)} players from ADP rankings")
            return True

        except Exception as e:
            print(f"❌ Direct file loading failed: {e}")

    # Try Method 3: Manual upload
    try:
        print("\n📤 Method 3: Manual File Upload")
        print("Please upload your CSV files:")
        from google.colab import files
        uploaded = files.upload()

        # Check what was uploaded
        if 'draft_board.csv' in uploaded and 'FantasyPros_2025_Overall_ADP_Rankings.csv' in uploaded:
            draft_board = pd.read_csv('draft_board.csv')
            adp_data = pd.read_csv('FantasyPros_2025_Overall_ADP_Rankings.csv')
            metadata = {'league_settings': {'teams': 12}, 'model_parameters': {}}

            print(f"✅ Uploaded and loaded {len(draft_board)} players from draft board")
            print(f"✅ Uploaded and loaded {len(adp_data)} players from ADP rankings")
            return True
        else:
            print("⚠️  Required CSV files not uploaded")

    except Exception as e:
        print(f"❌ Manual upload failed: {e}")

    # Try Method 4: Google Drive mount
    try:
        print("\n💾 Method 4: Google Drive Mount")
        from google.colab import drive
        drive.mount('/content/drive')

        # Common Google Drive paths
        drive_paths = [
            '/content/drive/MyDrive/fantasy-football/',
            '/content/drive/MyDrive/colab-data/',
            '/content/drive/MyDrive/'
        ]

        for drive_path in drive_paths:
            draft_path = os.path.join(drive_path, 'draft_board.csv')
            adp_path = os.path.join(drive_path, 'FantasyPros_2025_Overall_ADP_Rankings.csv')

            if os.path.exists(draft_path) and os.path.exists(adp_path):
                draft_board = pd.read_csv(draft_path)
                adp_data = pd.read_csv(adp_path)
                metadata = {'league_settings': {'teams': 12}, 'model_parameters': {}}

                print(f"✅ Loaded from Drive: {len(draft_board)} players from draft board")
                print(f"✅ Loaded from Drive: {len(adp_data)} players from ADP rankings")
                return True

        print("⚠️  CSV files not found in common Drive locations")

    except Exception as e:
        print(f"❌ Google Drive mount failed: {e}")

    # Method 5: Generate sample data
    print("\n🎲 Method 5: Using Sample Data")
    print("Creating sample data for demonstration...")

    # Generate realistic sample data
    sample_draft_data = []
    sample_adp_data = []

    # Sample top players with realistic stats
    top_players = [
        ('Christian McCaffrey', 'RB', 10.5, 21.3, 1),
        ('Tyreek Hill', 'WR', 9.8, 19.7, 2),
        ('Travis Kelce', 'TE', 8.9, 17.2, 3),
        ('Josh Allen', 'QB', 8.2, 22.1, 4),
        ('Austin Ekeler', 'RB', 7.8, 18.5, 5),
        ('Stefon Diggs', 'WR', 7.5, 17.8, 6),
        ('Davante Adams', 'WR', 7.2, 17.1, 7),
        ('Jonathan Taylor', 'RB', 6.9, 16.8, 8),
        ('Cooper Kupp', 'WR', 6.6, 16.3, 9),
        ('Derrick Henry', 'RB', 6.3, 15.9, 10)
    ]

    for i, (name, pos, vorp, ppg, adp) in enumerate(top_players):
        sample_draft_data.append({
            'player_name': name, 'position': pos, 'VORP': vorp,
            'proj_ppg_2025': ppg, 'adp_rank': adp
        })
        sample_adp_data.append({
            'player_name': name, 'position': pos, 'adp_rank': adp
        })

    # Add more players for realistic dataset
    for i in range(11, 200):
        pos = ['QB', 'RB', 'WR', 'TE', 'DEF', 'K'][i % 6]
        sample_draft_data.append({
            'player_name': f'Player_{i}', 'position': pos,
            'VORP': max(0, 8 - i*0.05), 'proj_ppg_2025': max(5, 20 - i*0.08),
            'adp_rank': i
        })
        sample_adp_data.append({
            'player_name': f'Player_{i}', 'position': pos, 'adp_rank': i
        })

    draft_board = pd.DataFrame(sample_draft_data)
    adp_data = pd.DataFrame(sample_adp_data)
    metadata = {'league_settings': {'teams': 12}, 'model_parameters': {}}

    print(f"✅ Generated sample data: {len(draft_board)} players")
    return True

# Execute the data loading
print("🔄 Starting data loading process...")
try:
    success = load_data_sources()
    if success and draft_board is not None and adp_data is not None:
        print(f"\n🎯 Data loading complete!")
        print(f"   📊 Draft board: {len(draft_board)} players")
        print(f"   📈 ADP data: {len(adp_data)} players")

        # Show data preview
        print(f"\n📋 Draft Board Preview:")
        print(draft_board.head(3).to_string())
        print(f"\n📈 ADP Data Preview:")
        print(adp_data.head(3).to_string())
    else:
        print("❌ Data loading failed!")

except Exception as e:
    print(f"❌ Critical error in data loading: {e}")
    print("📝 Please check your files and try again")

print("\n✅ Data loading section complete!")


🔄 Starting data loading process...
📦 Found colab_data.zip - extracting...
✅ ZIP file extracted successfully
🔄 Running colab_loader.py...
🔄 Loading data files...
✅ Data files detected!
📋 Loaded metadata
📊 Loaded draft_board.csv: 594 players
📈 Loaded ADP data: 378 players
🏈 Loaded rookie data: 501 players
🤖 Found rookie prediction model: rookie_regressor.pkl
🎯 All critical data loaded successfully!

🎯 Data loading complete!
   📊 Draft board: 594 players
   📈 ADP data: 378 players

📋 Draft Board Preview:
      player_name position  proj_ppg_2025       VORP  adp_rank
0  Saquon Barkley       RB      21.435000  10.676176       3.0
1    Jahmyr Gibbs       RB      20.188889   9.430065       4.0
2   Derrick Henry       RB      19.678947   8.920124       7.0

📈 ADP Data Preview:
   Rank          Player Team   Bye  POS  Yahoo  Sleeper  RTSports  AVG Real-Time (?)
0   1.0   Ja'Marr Chase  CIN  10.0  WR1    1.0      1.0       1.0  1.0             1
1   2.0  Bijan Robinson  ATL   5.0  RB1    2.0    

## 2. Data Structures and Game State


In [9]:
@dataclass
class Player:
    """Represents a fantasy football player"""
    name: str
    position: str
    team: str = ""
    vorp: float = 0.0
    proj_ppg: float = 0.0
    adp_rank: float = 999.0
    bye_week: int = 0
    risk_sigma: float = 0.0  # Uncertainty/variance for rookies
    is_rookie: bool = False

    def __hash__(self):
        return hash(self.name)

@dataclass
class LeagueSettings:
    """League configuration"""
    teams: int = 12
    roster_spots: Dict[str, int] = field(default_factory=lambda: {
        'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'FLEX': 1, 'DEF': 1, 'K': 1, 'BENCH': 6
    })
    flex_positions: Set[str] = field(default_factory=lambda: {'RB', 'WR', 'TE'})
    total_rounds: int = 15

    @property
    def total_roster_size(self):
        return sum(self.roster_spots.values())

@dataclass
class DraftState:
    """Current state of the draft"""
    league: LeagueSettings
    current_round: int = 1
    current_pick_in_round: int = 1
    available_players: Set[Player] = field(default_factory=set)
    team_rosters: Dict[int, List[Player]] = field(default_factory=lambda: defaultdict(list))
    our_team_id: int = 1

    @property
    def current_overall_pick(self):
        return (self.current_round - 1) * self.league.teams + self.current_pick_in_round

    @property
    def is_snake_draft(self):
        return True  # Assuming snake draft

    @property
    def current_team_picking(self):
        if self.current_round % 2 == 1:  # Odd rounds
            return self.current_pick_in_round
        else:  # Even rounds (snake)
            return self.league.teams - self.current_pick_in_round + 1

    @property
    def picks_until_our_turn(self):
        current_team = self.current_team_picking
        if current_team == self.our_team_id:
            return 0

        # Calculate picks until our next turn in snake draft
        if self.current_round % 2 == 1:  # Odd round
            if self.our_team_id > current_team:
                return self.our_team_id - current_team
            else:
                # Need to go through end of round and back
                picks_to_end = self.league.teams - current_team + 1
                picks_back = self.league.teams - self.our_team_id + 1
                return picks_to_end + picks_back
        else:  # Even round
            reverse_team = self.league.teams - current_team + 1
            reverse_our_team = self.league.teams - self.our_team_id + 1
            if reverse_our_team > reverse_team:
                return reverse_our_team - reverse_team
            else:
                # Need to go to next round
                picks_to_end = reverse_team
                return picks_to_end + self.our_team_id

    def get_roster_needs(self, team_id: int) -> Dict[str, int]:
        """Calculate remaining needs for a team"""
        roster = self.team_rosters[team_id]
        position_counts = Counter(p.position for p in roster)

        needs = {}
        for pos, required in self.league.roster_spots.items():
            if pos == 'FLEX':
                # FLEX can be filled by RB/WR/TE
                flex_filled = sum(max(0, position_counts.get(fp, 0) - self.league.roster_spots.get(fp, 0))
                                for fp in self.league.flex_positions)
                needs[pos] = max(0, required - flex_filled)
            elif pos == 'BENCH':
                total_players = len(roster)
                starting_spots = sum(v for k, v in self.league.roster_spots.items() if k != 'BENCH')
                needs[pos] = max(0, required - (total_players - starting_spots))
            else:
                needs[pos] = max(0, required - position_counts.get(pos, 0))

        return needs

    def is_draft_complete(self) -> bool:
        return self.current_round > self.league.total_rounds

    def copy(self):
        """Create a deep copy of the state"""
        return deepcopy(self)

# League settings
LEAGUE = LeagueSettings(
    teams=12,
    roster_spots={'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'FLEX': 1, 'DEF': 1, 'K': 1, 'BENCH': 6},
    total_rounds=15
)

print("✅ Data structures defined!")
print(f"🏈 League: {LEAGUE.teams} teams, {LEAGUE.total_rounds} rounds")
print(f"📋 Roster: {LEAGUE.roster_spots}")


✅ Data structures defined!
🏈 League: 12 teams, 15 rounds
📋 Roster: {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'FLEX': 1, 'DEF': 1, 'K': 1, 'BENCH': 6}


## 3. Data Processing and Player Pool Creation


### 3.1 Optional: Load Rookie Prediction Model


In [10]:
# Optional: Load rookie prediction model for better uncertainty estimates
rookie_model = None
rookie_scaler = None

try:
    # Try to load the rookie prediction model
    import pickle

    # Check multiple possible locations
    model_files = [
        'best_rookie_model.pkl',  # From rookie comparison notebook
        'rookie_fantasy_model.pkl',  # From rookie example
        '/content/best_rookie_model.pkl'  # If uploaded to Colab
    ]

    for model_file in model_files:
        if os.path.exists(model_file):
            print(f"📊 Loading rookie prediction model from {model_file}...")

            with open(model_file, 'rb') as f:
                rookie_model_data = pickle.load(f)

            rookie_model = rookie_model_data['model']
            rookie_scaler = rookie_model_data.get('scaler')
            rookie_features = rookie_model_data['feature_names']

            print(f"✅ Rookie model loaded: {rookie_model_data.get('model_name', 'Unknown')}")
            print(f"🎯 Model R²: {rookie_model_data.get('performance_metrics', {}).get('r2_mean', 'N/A')}")
            print(f"📋 Features: {len(rookie_features)}")
            break

    if rookie_model is None:
        print("⚠️  No rookie model found - will use simple uncertainty estimates")
        print("💡 To use rookie predictions:")
        print("   1. Run rookie_model_comparison.ipynb first")
        print("   2. Upload the generated best_rookie_model.pkl")
        print("   3. Restart this cell")

except Exception as e:
    print(f"❌ Error loading rookie model: {e}")
    print("📝 Using default uncertainty estimates")
    rookie_model = None

# Function to get rookie uncertainty using the ML model
def get_rookie_uncertainty(player_name: str, position: str, draft_info: dict = None) -> float:
    """
    Get uncertainty estimate for a rookie using the ML model

    Args:
        player_name: Player name
        position: Player position
        draft_info: Dict with draft capital info (round, pick, age, etc.)

    Returns:
        Uncertainty value (higher = more risky)
    """

    if rookie_model is None or draft_info is None:
        # Default uncertainty based on position
        default_uncertainty = {
            'QB': 3.0,  # High variance for rookie QBs
            'RB': 2.0,  # Moderate variance for RBs
            'WR': 2.5,  # Moderate-high variance for WRs
            'TE': 1.5,  # Lower variance for TEs
            'DEF': 0.5, # Low variance for defense
            'K': 0.3    # Very low variance for kickers
        }
        return default_uncertainty.get(position, 1.5)

    try:
        # Create feature vector for rookie model prediction
        rookie_features_dict = {
            'round': draft_info.get('round', 4),
            'pick': draft_info.get('pick', 100),
            'age': draft_info.get('age', 22),
            'early_round': 1 if draft_info.get('round', 4) <= 3 else 0,
            'first_round': 1 if draft_info.get('round', 4) == 1 else 0,
            'day1_pick': 1 if draft_info.get('pick', 100) <= 32 else 0,
            'day2_pick': 1 if 32 < draft_info.get('pick', 100) <= 96 else 0,
            'is_qb': 1 if position == 'QB' else 0,
            'is_rb': 1 if position == 'RB' else 0,
            'is_wr': 1 if position == 'WR' else 0,
            'is_te': 1 if position == 'TE' else 0,
            'good_team': draft_info.get('good_team', 0),
            'good_offense': draft_info.get('good_offense', 0),
            'bad_offense': draft_info.get('bad_offense', 0),
            'games_played_pct': 0.8,  # Assume 80% games played
            'target_share': 0.1 if position in ['WR', 'TE'] else 0.0,
            'rush_share': 0.15 if position == 'RB' else 0.0,
            'starter_games': 1 if draft_info.get('round', 4) <= 3 else 0,
            'yards_per_target': 8.0 if position in ['WR', 'TE'] else 0.0,
            'yards_per_carry': 4.0 if position == 'RB' else 0.0
        }

        # Convert to dataframe with correct feature order
        import pandas as pd
        feature_df = pd.DataFrame([rookie_features_dict])

        # Only use features that the model was trained on
        available_features = [f for f in rookie_features if f in feature_df.columns]
        X = feature_df[available_features].fillna(0)

        # Scale if needed
        if rookie_scaler is not None:
            X_scaled = rookie_scaler.transform(X)
        else:
            X_scaled = X

        # Get prediction - this gives us expected PPG
        predicted_ppg = rookie_model.predict(X_scaled)[0]

        # Convert prediction confidence to uncertainty
        # Higher predicted PPG = lower uncertainty for established talent
        # But rookies always have base uncertainty
        base_uncertainty = 1.0  # Minimum uncertainty for rookies

        # Draft capital uncertainty - later picks = more uncertain
        draft_uncertainty = max(0, (draft_info.get('round', 4) - 1) * 0.3)

        # Position-specific uncertainty
        position_uncertainty = {
            'QB': 1.5, 'RB': 1.0, 'WR': 1.2, 'TE': 0.8, 'DEF': 0.3, 'K': 0.2
        }.get(position, 1.0)

        total_uncertainty = base_uncertainty + draft_uncertainty + position_uncertainty

        return min(total_uncertainty, 4.0)  # Cap at 4.0

    except Exception as e:
        print(f"⚠️  Error in rookie prediction for {player_name}: {e}")
        # Fallback to position-based default
        return {
            'QB': 3.0, 'RB': 2.0, 'WR': 2.5, 'TE': 1.5, 'DEF': 0.5, 'K': 0.3
        }.get(position, 1.5)

print("✅ Rookie uncertainty estimation ready!")
if rookie_model:
    print("🤖 Using ML model for rookie predictions")
else:
    print("📊 Using position-based uncertainty estimates")


⚠️  No rookie model found - will use simple uncertainty estimates
💡 To use rookie predictions:
   1. Run rookie_model_comparison.ipynb first
   2. Upload the generated best_rookie_model.pkl
   3. Restart this cell
✅ Rookie uncertainty estimation ready!
📊 Using position-based uncertainty estimates


In [13]:
def create_player_pool() -> Dict[str, Player]:
    """Create unified player pool from draft board and ADP data"""

    # Clean and standardize column names
    board = draft_board.copy()
    adp = adp_data.copy()

    # Standardize column names
    if 'Player' in adp.columns:
        adp = adp.rename(columns={'Player': 'player_name'})
    if 'POS' in adp.columns:
        adp = adp.rename(columns={'POS': 'position'})
    if 'Yahoo' in adp.columns:
        adp = adp.rename(columns={'Yahoo': 'adp_rank'})

    # Clean position data (remove numbers like WR1 -> WR)
    if 'position' in board.columns:
        board['position'] = board['position'].str.replace(r'\d+', '', regex=True)
    if 'position' in adp.columns:
        adp['position'] = adp['position'].str.replace(r'\d+', '', regex=True)

    # Map D/ST to DEF
    position_mapping = {'D/ST': 'DEF', 'DST': 'DEF'}
    for df in [board, adp]:
        if 'position' in df.columns:
            df['position'] = df['position'].replace(position_mapping)

    players = {}

    # Process draft board players
    for _, row in board.iterrows():
        name = str(row.get('player_name', '')).strip()
        if not name or name == 'nan':
            continue

        player = Player(
            name=name,
            position=str(row.get('position', 'UNKNOWN')),
            vorp=float(row.get('VORP', 0.0)),
            proj_ppg=float(row.get('proj_ppg_2025', 0.0)),
            adp_rank=float(row.get('adp_rank', 999.0)),
            risk_sigma=float(row.get('proj_ppg_2025', 0.0)) * 0.1  # 10% uncertainty as default
        )
        players[name] = player

    # Update with ADP data
    for _, row in adp.iterrows():
        name = str(row.get('player_name', '')).strip()
        if not name or name == 'nan':
            continue

        adp_rank = float(row.get('adp_rank', 999.0))
        position = str(row.get('position', 'UNKNOWN'))

        if name in players:
            # Update existing player
            if adp_rank < 999:
                players[name].adp_rank = adp_rank
            if position != 'UNKNOWN':
                players[name].position = position
        else:
            # Create new player from ADP
            players[name] = Player(
                name=name,
                position=position,
                adp_rank=adp_rank,
                vorp=0.0,      # Will be estimated later
                proj_ppg=8.0,  # Default projection
                risk_sigma=1.0 # Higher uncertainty for unknown players
            )

    # Estimate VORP for players without it using positional averages
    position_vorp_avg = {}
    for player in players.values():
        if player.vorp > 0:
            if player.position not in position_vorp_avg:
                position_vorp_avg[player.position] = []
            position_vorp_avg[player.position].append(player.vorp)

    # Calculate averages
    for pos in position_vorp_avg:
        position_vorp_avg[pos] = np.mean(position_vorp_avg[pos])

    # Fill missing VORP values
    for player in players.values():
        if player.vorp <= 0 and player.position in position_vorp_avg:
            # Estimate based on ADP rank within position
            pos_players = [
                p for p in players.values()
                if p.position == player.position and p.adp_rank < 999
            ]
            if pos_players:
                pos_players.sort(key=lambda x: x.adp_rank)
                player_rank = next(
                    (i for i, p in enumerate(pos_players) if p.name == player.name),
                    len(pos_players)
                )
                # Decay VORP by rank
                base_vorp = position_vorp_avg[player.position]
                player.vorp = base_vorp * (0.9 ** (player_rank / 10))  # Exponential decay

    # Filter out players with no meaningful data
    valid_players = {
        name: player for name, player in players.items()
        if player.adp_rank < 300 or player.vorp > 0
    }

    print(f"✅ Created player pool with {len(valid_players)} players")

    # Show position breakdown
    pos_counts = Counter(p.position for p in valid_players.values())
    print(f"📊 Position breakdown: {dict(pos_counts)}")

    return valid_players


# Create the player pool (uses globals if you don't pass args)
player_pool = create_player_pool()

# Show top players by VORP
top_players = sorted(player_pool.values(), key=lambda x: x.vorp, reverse=True)[:50]
print(f"\n🏆 Top 20 Players by VORP:")
for i, player in enumerate(top_players, 1):
    print(f"{i:2d}. {player.name:25s} {player.position:3s} VORP:{player.vorp:6.2f} ADP:{player.adp_rank:6.1f}")






✅ Created player pool with 755 players
📊 Position breakdown: {'RB': 170, 'WR': 272, 'QB': 99, 'TE': 131, 'DEF': 62, 'K': 21}

🏆 Top 20 Players by VORP:
 1. Saquon Barkley            RB  VORP: 10.68 ADP:   3.0
 2. Jahmyr Gibbs              RB  VORP:  9.43 ADP:   4.0
 3. Derrick Henry             RB  VORP:  8.92 ADP:   7.0
 4. Ja'Marr Chase             WR  VORP:  8.50 ADP:   1.0
 5. Bijan Robinson            RB  VORP:  7.43 ADP:   2.0
 6. Lamar Jackson             QB  VORP:  6.62 ADP:  21.0
 7. Jonathan Taylor           RB  VORP:  6.22 ADP:  17.0
 8. Rashee Rice               WR  VORP:  6.17 ADP:  64.0
 9. Josh Jacobs               RB  VORP:  5.95 ADP:  16.0
10. Alvin Kamara              RB  VORP:  5.76 ADP:  45.0
11. George Kittle             TE  VORP:  5.71 ADP:  30.0
12. Joe Mixon                 RB  VORP:  5.36 ADP:  74.0
13. James Cook                RB  VORP:  5.31 ADP:  31.0
14. Kyren Williams            RB  VORP:  5.29 ADP:  26.0
15. Brock Bowers              TE  VORP:  4.70 ADP:

In [14]:
# Enhanced rookie uncertainty estimation using ML model
print("🔄 Applying enhanced rookie uncertainty estimates...")

rookie_count = 0
veteran_count = 0

for player_name, player in player_pool.items():
    # Determine if player is likely a rookie (simple heuristic)
    is_rookie = (
        'rookie' in player_name.lower() or
        player.adp_rank > 150 or  # Late ADP often indicates rookie
        player.vorp < 1.0  # Low VORP might indicate rookie uncertainty
    )

    # Update player rookie status and uncertainty
    player.is_rookie = is_rookie

    if is_rookie:
        # Use ML model for rookie uncertainty if available
        draft_info = {
            'round': 4,  # Default round (will be updated with real data if available)
            'pick': min(player.adp_rank, 200),  # Use ADP as proxy for draft pick
            'age': 22,   # Default rookie age
            'good_team': 0,  # Default neutral team
            'good_offense': 0,
            'bad_offense': 0
        }

        # Try to get draft info from data if available
        if hasattr(player, 'round'):
            draft_info['round'] = player.round
        elif player.adp_rank <= 32:
            draft_info['round'] = 1
        elif player.adp_rank <= 64:
            draft_info['round'] = 2
        elif player.adp_rank <= 96:
            draft_info['round'] = 3

        enhanced_uncertainty = get_rookie_uncertainty(player_name, player.position, draft_info)
        player.risk_sigma = enhanced_uncertainty
        rookie_count += 1
    else:
        # Veterans get lower uncertainty (5% of projection)
        player.risk_sigma = max(player.proj_ppg * 0.05, 0.3)
        veteran_count += 1

print(f"✅ Enhanced uncertainty applied:")
print(f"   🆕 Rookies: {rookie_count} players")
print(f"   👥 Veterans: {veteran_count} players")

# Show examples of uncertainty assignments
print(f"\n📊 Sample uncertainty assignments:")
sample_players = sorted(player_pool.values(), key=lambda x: x.vorp, reverse=True)[:10]
for i, player in enumerate(sample_players):
    rookie_indicator = "🆕" if player.is_rookie else "👥"
    print(f"   {rookie_indicator} {player.name:<25s} ({player.position}) - Uncertainty: {player.risk_sigma:.2f}")

if rookie_model:
    print(f"\n🤖 Using ML model for rookie predictions!")
else:
    print(f"\n📊 Using heuristic-based uncertainty estimates")


🔄 Applying enhanced rookie uncertainty estimates...
✅ Enhanced uncertainty applied:
   🆕 Rookies: 326 players
   👥 Veterans: 429 players

📊 Sample uncertainty assignments:
   👥 Saquon Barkley            (RB) - Uncertainty: 1.07
   👥 Jahmyr Gibbs              (RB) - Uncertainty: 1.01
   👥 Derrick Henry             (RB) - Uncertainty: 0.98
   👥 Ja'Marr Chase             (WR) - Uncertainty: 1.00
   👥 Bijan Robinson            (RB) - Uncertainty: 0.91
   👥 Lamar Jackson             (QB) - Uncertainty: 1.28
   👥 Jonathan Taylor           (RB) - Uncertainty: 0.85
   👥 Rashee Rice               (WR) - Uncertainty: 0.88
   👥 Josh Jacobs               (RB) - Uncertainty: 0.84
   👥 Alvin Kamara              (RB) - Uncertainty: 0.83

📊 Using heuristic-based uncertainty estimates


## 4. Opponent Modeling with Plackett-Luce


In [16]:
class OpponentModel:
    """Models how opponents draft using ADP with stochastic sampling"""

    def __init__(self, player_pool: Dict[str, Player], temperature: float = 0.5,
                 position_run_prob: float = 0.1):
        self.player_pool = player_pool
        self.temperature = temperature  # Higher = more random
        self.position_run_prob = position_run_prob  # Probability of position runs
        self.position_run_active = None  # Currently active position run
        self.position_run_remaining = 0

        # Create ADP-based preference weights
        self.player_weights = {}
        for player in player_pool.values():
            if player.adp_rank < 999:
                # Convert ADP rank to weight (lower rank = higher weight)
                # Use exponential decay: higher picks much more likely
                self.player_weights[player.name] = np.exp(-player.adp_rank / 50.0)
            else:
                self.player_weights[player.name] = 0.001  # Very low weight for unranked

    def sample_opponent_pick(self, state: DraftState, team_id: int) -> Optional[Player]:
        """Sample a pick for an opponent team using Plackett-Luce model"""

        available = list(state.available_players)
        if not available:
            return None

        # Get team's positional needs
        needs = state.get_roster_needs(team_id)

        # Calculate selection probabilities
        probs = []
        for player in available:
            # Base weight from ADP
            weight = self.player_weights.get(player.name, 0.001)\n            \n            # Position need bonus\n            need_bonus = 1.0\n            if player.position in needs and needs[player.position] > 0:\n                need_bonus = 2.0  # 2x bonus for needed positions\n            elif player.position in state.league.flex_positions and needs.get('FLEX', 0) > 0:\n                need_bonus = 1.5  # 1.5x bonus for flex-eligible positions\n            \n            # Position run bonus\n            run_bonus = 1.0\n            if (self.position_run_active == player.position and \n                self.position_run_remaining > 0):\n                run_bonus = 3.0\n            \n            # Don't draft too many of same position early\n            roster = state.team_rosters[team_id]\n            pos_count = sum(1 for p in roster if p.position == player.position)\n            max_reasonable = state.league.roster_spots.get(player.position, 1) + 1\n            if pos_count >= max_reasonable and state.current_round <= 8:\n                weight *= 0.1  # Heavy penalty for overdrafting early\n            \n            final_weight = weight * need_bonus * run_bonus\n            probs.append(final_weight)\n        \n        # Apply temperature for randomness\n        if self.temperature > 0:\n            log_probs = np.log(np.array(probs) + 1e-10)\n            scaled_log_probs = log_probs / self.temperature\n            probs = np.exp(scaled_log_probs - np.max(scaled_log_probs))  # Numerical stability\n        \n        # Normalize probabilities\n        probs = np.array(probs)\n        probs = probs / np.sum(probs)\n        \n        # Sample player\n        selected_idx = np.random.choice(len(available), p=probs)\n        selected_player = available[selected_idx]\n        \n        # Update position run state\n        if self.position_run_remaining > 0:\n            self.position_run_remaining -= 1\n            if self.position_run_remaining == 0:\n                self.position_run_active = None\n        else:\n            # Potentially start a new position run\n            if (np.random.random() < self.position_run_prob and \n                selected_player.position in ['RB', 'WR', 'QB']):\n                self.position_run_active = selected_player.position\n                self.position_run_remaining = np.random.randint(2, 5)  # 2-4 more picks\n        \n        return selected_player\n    \n    def reset_position_runs(self):\n        \"\"\"Reset position run state (call at start of new simulation)\"\"\"\n        self.position_run_active = None\n        self.position_run_remaining = 0\n\n# Create opponent model\nopponent_model = OpponentModel(player_pool, temperature=0.5, position_run_prob=0.1)\n\nprint(\"✅ Opponent model created!\")\nprint(f\"🎲 Temperature: {opponent_model.temperature} (higher = more random)\")\nprint(f\"🏃 Position run probability: {opponent_model.position_run_prob}\")\n\n# Test the opponent model\nprint(\"\\n🧪 Testing opponent model with sample picks:\")\ntest_state = DraftState(league=LEAGUE, available_players=set(player_pool.values()))\nfor i in range(5):\n    pick = opponent_model.sample_opponent_pick(test_state, team_id=2)\n    if pick:\n        print(f\"Pick {i+1}: {pick.name} ({pick.position}) - ADP: {pick.adp_rank:.1f}, VORP: {pick.vorp:.2f}\")\n        test_state.available_players.remove(pick)\n        test_state.team_rosters[2].append(pick)"


SyntaxError: unexpected character after line continuation character (ipython-input-3656458357.py, line 36)

## 5. Reward Function and Value Estimation


In [ ]:
class RewardFunction:
    """Calculates rewards for draft decisions"""

    def __init__(self, risk_penalty: float = 0.1, overstack_penalty: float = 0.5,
                 bye_penalty: float = 0.2):
        self.risk_penalty = risk_penalty  # λ - penalty for player uncertainty
        self.overstack_penalty = overstack_penalty  # γ - penalty for position overstacking
        self.bye_penalty = bye_penalty  # β - penalty for bye week conflicts

    def calculate_roster_value(self, roster: List[Player], league: LeagueSettings) -> float:
        """Calculate total value of a roster"""
        if not roster:
            return 0.0

        # Base VORP sum
        base_vorp = sum(player.vorp for player in roster)

        # Risk penalty (uncertainty, especially for rookies)
        risk_penalty = self.risk_penalty * sum(player.risk_sigma for player in roster)

        # Position overstacking penalty
        position_counts = Counter(p.position for p in roster)
        overstack_penalty = 0.0
        for pos, count in position_counts.items():
            max_useful = league.roster_spots.get(pos, 0)
            if pos in league.flex_positions:
                max_useful += league.roster_spots.get('FLEX', 0)
            max_useful += 2  # Allow some bench depth

            if count > max_useful:
                overstack_penalty += self.overstack_penalty * (count - max_useful) ** 2

        # Bye week conflict penalty (simplified)
        bye_weeks = [p.bye_week for p in roster if p.bye_week > 0]
        bye_conflicts = len(bye_weeks) - len(set(bye_weeks))
        bye_penalty = self.bye_penalty * bye_conflicts

        total_value = base_vorp - risk_penalty - overstack_penalty - bye_penalty
        return total_value

    def calculate_pick_reward(self, player: Player, current_roster: List[Player],
                             league: LeagueSettings) -> float:
        """Calculate incremental reward for picking a specific player"""

        # Base VORP
        base_reward = player.vorp

        # Position need bonus
        position_counts = Counter(p.position for p in current_roster)
        needs = {}
        for pos, required in league.roster_spots.items():
            needs[pos] = max(0, required - position_counts.get(pos, 0))

        need_bonus = 0.0
        if player.position in needs and needs[player.position] > 0:
            need_bonus = player.vorp * 0.2  # 20% bonus for needed positions
        elif player.position in league.flex_positions and needs.get('FLEX', 0) > 0:
            need_bonus = player.vorp * 0.1  # 10% bonus for flex-eligible

        # Risk penalty
        risk_penalty = self.risk_penalty * player.risk_sigma

        # Early round scarcity bonus
        round_num = len(current_roster) // 12 + 1  # Approximate round
        scarcity_bonus = 0.0
        if round_num <= 3 and player.position in ['RB', 'WR']:
            scarcity_bonus = player.vorp * 0.15  # 15% bonus for scarce positions early

        total_reward = base_reward + need_bonus + scarcity_bonus - risk_penalty
        return total_reward

class ValueFunction:
    """Heuristic value function for MCTS"""

    def __init__(self, player_pool: Dict[str, Player], reward_fn: RewardFunction):
        self.player_pool = player_pool
        self.reward_fn = reward_fn

        # Precompute position-ranked players for efficiency
        self.players_by_position = defaultdict(list)
        for player in player_pool.values():
            self.players_by_position[player.position].append(player)

        # Sort by VORP descending
        for pos in self.players_by_position:
            self.players_by_position[pos].sort(key=lambda x: x.vorp, reverse=True)

    def estimate_state_value(self, state: DraftState) -> float:
        """Estimate the value of the current state for our team"""

        our_roster = state.team_rosters[state.our_team_id]

        # Current roster value
        current_value = self.reward_fn.calculate_roster_value(our_roster, state.league)

        # Expected value of filling remaining roster spots
        expected_fill_value = self._estimate_expected_fill_value(state)

        return current_value + expected_fill_value

    def _estimate_expected_fill_value(self, state: DraftState) -> float:
        """Estimate value of filling remaining roster spots"""

        our_roster = state.team_rosters[state.our_team_id]
        needs = state.get_roster_needs(state.our_team_id)
        available = list(state.available_players)

        if not available:
            return 0.0

        # Estimate picks until our next few turns
        picks_until_next = state.picks_until_our_turn
        if picks_until_next == 0:
            picks_until_next = state.league.teams  # Next turn after this one

        # Estimate how many players will be taken before our next pick
        total_remaining_picks = sum(needs.values())
        if total_remaining_picks == 0:
            return 0.0

        expected_value = 0.0
        pick_number = 0

        for position, needed in needs.items():
            if needed == 0:
                continue

            available_at_pos = [p for p in available if p.position == position]
            if position == 'FLEX':
                available_at_pos = [p for p in available if p.position in state.league.flex_positions]
            elif position == 'BENCH':
                available_at_pos = available  # Any position for bench

            if not available_at_pos:
                continue

            available_at_pos.sort(key=lambda x: x.vorp, reverse=True)

            for i in range(min(needed, len(available_at_pos))):
                # Estimate which player we might get based on pick timing
                # Assume some players will be taken before our turn
                estimated_pick_index = min(pick_number + picks_until_next // 2,
                                         len(available_at_pos) - 1)

                if estimated_pick_index < len(available_at_pos):
                    expected_player = available_at_pos[estimated_pick_index]
                    expected_value += expected_player.vorp * 0.8  # Discount for uncertainty

                pick_number += 1
                picks_until_next = max(state.league.teams, picks_until_next)  # Future picks farther apart

        return expected_value

# Create reward function and value estimator
reward_function = RewardFunction(risk_penalty=0.1, overstack_penalty=0.5, bye_penalty=0.2)
value_function = ValueFunction(player_pool, reward_function)

print("✅ Reward and value functions created!")
print(f"🎯 Risk penalty (λ): {reward_function.risk_penalty}")
print(f"🚫 Overstack penalty (γ): {reward_function.overstack_penalty}")
print(f"📅 Bye week penalty (β): {reward_function.bye_penalty}")

# Test the reward function
test_roster = [
    next(p for p in player_pool.values() if p.position == 'RB'),
    next(p for p in player_pool.values() if p.position == 'WR'),
    next(p for p in player_pool.values() if p.position == 'QB')
]

test_value = reward_function.calculate_roster_value(test_roster, LEAGUE)
print(f"\n🧪 Test roster value: {test_value:.2f}")
for player in test_roster:
    reward = reward_function.calculate_pick_reward(player, test_roster[:-1], LEAGUE)
    print(f"   {player.name} ({player.position}): {reward:.2f} reward")


## 6. MCTS Implementation


In [ ]:
# Simplified MCTS implementation for draft strategy
class MCTSNode:
    """Node in the MCTS tree"""
    def __init__(self, state: DraftState, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.action = action  # Player that was picked to reach this state
        self.children = {}
        self.visits = 0
        self.value_sum = 0.0
        self.untried_actions = None

    @property
    def is_fully_expanded(self):
        return len(self.untried_actions) == 0 if self.untried_actions is not None else False

    @property
    def is_terminal(self):
        return self.state.is_draft_complete()

    def ucb1_score(self, exploration_constant=1.414):
        """Calculate UCB1 score for action selection"""
        if self.visits == 0:
            return float('inf')

        exploitation = self.value_sum / self.visits
        exploration = exploration_constant * math.sqrt(math.log(self.parent.visits) / self.visits)
        return exploitation + exploration

class SimplifiedMCTS:
    """Simplified MCTS for draft decisions"""

    def __init__(self, opponent_model: OpponentModel, value_function: ValueFunction,
                 reward_function: RewardFunction, simulations=400):
        self.opponent_model = opponent_model
        self.value_function = value_function
        self.reward_function = reward_function
        self.simulations = simulations

    def search(self, initial_state: DraftState) -> Player:
        """Run MCTS to find best action"""
        root = MCTSNode(initial_state.copy())

        for _ in range(self.simulations):
            # Selection and expansion
            node = self._select_and_expand(root)

            # Simulation (rollout)
            value = self._rollout(node.state.copy())

            # Backpropagation
            self._backpropagate(node, value)

        # Return action with most visits (most robust)
        if not root.children:
            # Fallback to greedy VORP if no expansions
            available = list(initial_state.available_players)
            if available:
                return max(available, key=lambda p: p.vorp)
            return None

        best_action = max(root.children.keys(), key=lambda a: root.children[a].visits)
        return best_action

    def _select_and_expand(self, root: MCTSNode) -> MCTSNode:
        """Select path through tree and expand if possible"""
        node = root

        # Selection: traverse tree using UCB1
        while not node.is_terminal and node.is_fully_expanded:
            if not node.children:
                break
            node = max(node.children.values(), key=lambda n: n.ucb1_score())

        # Expansion: add new child if possible
        if not node.is_terminal:
            if node.untried_actions is None:
                node.untried_actions = self._get_valid_actions(node.state)

            if node.untried_actions:
                action = node.untried_actions.pop()
                new_state = self._apply_action(node.state.copy(), action)
                child = MCTSNode(new_state, parent=node, action=action)
                node.children[action] = child
                return child

        return node

    def _get_valid_actions(self, state: DraftState) -> List[Player]:
        """Get valid actions (available players) with action masking"""
        available = list(state.available_players)

        # Action masking: remove clearly bad picks
        our_roster = state.team_rosters[state.our_team_id]
        position_counts = Counter(p.position for p in our_roster)

        valid_actions = []
        for player in available:
            # Don't draft 3rd QB early
            if (player.position == 'QB' and position_counts.get('QB', 0) >= 2 and
                state.current_round <= 10):
                continue

            # Don't draft 4th RB/WR early unless exceptional value
            if (player.position in ['RB', 'WR'] and position_counts.get(player.position, 0) >= 3 and
                state.current_round <= 8 and player.vorp < 3.0):
                continue

            # Don't draft DEF/K too early
            if (player.position in ['DEF', 'K'] and state.current_round <= 12):
                continue

            valid_actions.append(player)

        # Sort by VORP for better action ordering
        valid_actions.sort(key=lambda p: p.vorp, reverse=True)
        return valid_actions[:50]  # Limit to top 50 to keep search manageable

    def _apply_action(self, state: DraftState, action: Player) -> DraftState:
        """Apply action (pick player) to state"""
        # Remove player from available pool
        state.available_players.discard(action)

        # Add to our roster
        state.team_rosters[state.our_team_id].append(action)

        # Advance to next pick
        if state.current_pick_in_round < state.league.teams:
            state.current_pick_in_round += 1
        else:
            state.current_round += 1
            state.current_pick_in_round = 1

        return state

    def _rollout(self, state: DraftState) -> float:
        """Simulate rest of draft using simple policy"""
        rollout_state = state.copy()

        while not rollout_state.is_draft_complete() and rollout_state.available_players:
            current_team = rollout_state.current_team_picking

            if current_team == rollout_state.our_team_id:
                # Our turn: use greedy VORP with positional needs
                available = list(rollout_state.available_players)
                if not available:
                    break

                our_roster = rollout_state.team_rosters[rollout_state.our_team_id]
                best_player = None
                best_score = -float('inf')

                for player in available[:20]:  # Consider top 20 for speed
                    score = self.reward_function.calculate_pick_reward(player, our_roster, rollout_state.league)
                    if score > best_score:
                        best_score = score
                        best_player = player

                if best_player:
                    rollout_state.available_players.discard(best_player)
                    rollout_state.team_rosters[current_team].append(best_player)
            else:
                # Opponent turn: use opponent model
                pick = self.opponent_model.sample_opponent_pick(rollout_state, current_team)
                if pick:
                    rollout_state.available_players.discard(pick)
                    rollout_state.team_rosters[current_team].append(pick)

            # Advance pick
            if rollout_state.current_pick_in_round < rollout_state.league.teams:
                rollout_state.current_pick_in_round += 1
            else:
                rollout_state.current_round += 1
                rollout_state.current_pick_in_round = 1

        # Evaluate final state
        return self.value_function.estimate_state_value(rollout_state)

    def _backpropagate(self, node: MCTSNode, value: float):
        """Backpropagate value up the tree"""
        while node is not None:
            node.visits += 1
            node.value_sum += value
            node = node.parent

# Create MCTS agent
mcts_agent = SimplifiedMCTS(opponent_model, value_function, reward_function, simulations=200)

print("✅ MCTS agent created!")
print(f"🔍 Simulations per decision: {mcts_agent.simulations}")
print("🎯 Ready for draft strategy testing!")


## 7. Backtesting Framework - Strategy Comparison


### 6.1 GPU-Accelerated MCTS (T4 Optimized)


In [ ]:
if GPU_AVAILABLE:

    class GPUAcceleratedMCTS:
        """
        GPU-optimized MCTS implementation using PyTorch and CuPy
        Designed specifically for T4 GPU with batch processing
        """

        def __init__(self, opponent_model, reward_function, value_function,
                     c_param=1.4, simulations_per_move=800, batch_size=64):
            self.opponent_model = opponent_model
            self.reward_function = reward_function
            self.value_function = value_function
            self.c_param = c_param
            self.simulations_per_move = simulations_per_move
            self.batch_size = batch_size  # T4-optimized batch size

            # GPU tensors for fast computation
            self.device = torch.device('cuda')

            # Pre-allocate GPU memory for common computations
            self._init_gpu_buffers()

        def _init_gpu_buffers(self):
            """Pre-allocate GPU memory buffers for efficiency"""
            max_players = 500  # Reasonable upper bound
            max_simulations = 2000

            # Pre-allocate tensors
            self.player_values_gpu = torch.zeros(max_players, device=self.device, dtype=torch.float32)
            self.ucb_scores_gpu = torch.zeros(max_players, device=self.device, dtype=torch.float32)
            self.visit_counts_gpu = torch.zeros(max_players, device=self.device, dtype=torch.int32)
            self.win_rates_gpu = torch.zeros(max_players, device=self.device, dtype=torch.float32)

            print(f"🚀 GPU buffers allocated on {self.device}")

        @torch.jit.script_method if hasattr(torch.jit, 'script_method') else lambda x: x
        def _calculate_ucb_batch(self, win_rates: torch.Tensor, visit_counts: torch.Tensor,
                                total_visits: int, c_param: float) -> torch.Tensor:
            """Vectorized UCB calculation on GPU"""
            if total_visits == 0:
                return torch.full_like(win_rates, float('inf'))

            exploration = c_param * torch.sqrt(
                torch.log(torch.tensor(total_visits, device=win_rates.device)) /
                torch.clamp(visit_counts.float(), min=1.0)
            )

            return win_rates + exploration

        def _batch_simulate_rollouts(self, states: List, batch_size: int = None) -> torch.Tensor:
            """
            Batch process multiple rollout simulations on GPU
            """
            if batch_size is None:
                batch_size = min(self.batch_size, len(states))

            results = []

            for i in range(0, len(states), batch_size):
                batch_states = states[i:i + batch_size]
                batch_results = []

                # Process batch of rollouts
                for state in batch_states:
                    # Use value function for fast approximation
                    value = self.value_function.estimate_state_value(state)
                    batch_results.append(value)

                # Convert to GPU tensor
                batch_tensor = torch.tensor(batch_results, device=self.device, dtype=torch.float32)
                results.append(batch_tensor)

            return torch.cat(results) if results else torch.tensor([], device=self.device)

        def _gpu_action_selection(self, available_actions: List, node_stats: Dict) -> int:
            """
            GPU-accelerated action selection using vectorized UCB
            """
            if not available_actions:
                return None

            n_actions = len(available_actions)

            # Prepare data on GPU
            win_rates = torch.zeros(n_actions, device=self.device)
            visit_counts = torch.zeros(n_actions, device=self.device)

            total_visits = sum(node_stats.get(action, {}).get('visits', 0) for action in available_actions)

            for i, action in enumerate(available_actions):
                stats = node_stats.get(action, {'wins': 0, 'visits': 0})
                visit_counts[i] = stats['visits']
                win_rates[i] = stats['wins'] / max(stats['visits'], 1)

            # Calculate UCB scores on GPU
            ucb_scores = self._calculate_ucb_batch(win_rates, visit_counts, total_visits, self.c_param)

            # Select best action
            best_idx = torch.argmax(ucb_scores).item()
            return available_actions[best_idx]

        def search(self, root_state: 'DraftState') -> int:
            """
            Main MCTS search with GPU acceleration
            """
            # Statistics tracking
            stats = defaultdict(lambda: {'wins': 0, 'visits': 0})

            # Batch simulations for GPU efficiency
            simulation_batches = self.simulations_per_move // self.batch_size
            remainder = self.simulations_per_move % self.batch_size

            print(f"🔥 Running {self.simulations_per_move} GPU-accelerated simulations...")

            with torch.cuda.amp.autocast():  # Mixed precision for T4 efficiency

                for batch_idx in tqdm(range(simulation_batches), desc="GPU Batches"):
                    current_batch_size = self.batch_size
                    if batch_idx == simulation_batches - 1:
                        current_batch_size += remainder

                    # Selection phase - find promising actions
                    available_actions = root_state.get_valid_actions()
                    if not available_actions:
                        break

                    # Batch simulate multiple paths
                    batch_states = []
                    batch_actions = []

                    for _ in range(current_batch_size):
                        # Select action using GPU-accelerated UCB
                        action = self._gpu_action_selection(available_actions, stats)
                        if action is None:
                            continue

                        # Apply action to get new state
                        new_state = root_state.copy()
                        new_state.make_pick(action)

                        batch_states.append(new_state)
                        batch_actions.append(action)

                    if not batch_states:
                        continue

                    # Batch rollout evaluation on GPU
                    rollout_values = self._batch_simulate_rollouts(batch_states, current_batch_size)

                    # Backpropagation
                    for action, value in zip(batch_actions, rollout_values.cpu().numpy()):
                        stats[action]['visits'] += 1
                        stats[action]['wins'] += float(value)

            # Select final action
            if not stats:
                available_actions = root_state.get_valid_actions()
                return available_actions[0] if available_actions else None

            # Return action with highest win rate
            best_action = max(stats.keys(), key=lambda a: stats[a]['wins'] / max(stats[a]['visits'], 1))

            # Print GPU performance stats
            total_simulations = sum(stats[a]['visits'] for a in stats)
            print(f"✅ Completed {total_simulations} GPU simulations")
            print(f"🎯 Best action win rate: {stats[best_action]['wins'] / stats[best_action]['visits']:.3f}")

            return best_action

    print("✅ GPU-Accelerated MCTS ready for T4!")

else:
    print("⚠️  GPU not available - using CPU MCTS")


In [ ]:
class AdaptiveMCTSStrategy(DraftStrategy):
    """
    Adaptive MCTS Strategy that automatically uses GPU acceleration when available
    Falls back to CPU implementation for compatibility
    """

    def __init__(self, opponent_model, reward_function, value_function,
                 simulations_per_move=800, c_param=1.4, use_gpu=True):
        super().__init__("Adaptive MCTS")
        self.opponent_model = opponent_model
        self.reward_function = reward_function
        self.value_function = value_function
        self.simulations_per_move = simulations_per_move
        self.c_param = c_param
        self.use_gpu = use_gpu and GPU_AVAILABLE

        # Initialize the appropriate MCTS implementation
        if self.use_gpu:
            print("🚀 Using GPU-Accelerated MCTS")
            self.mcts = GPUAcceleratedMCTS(
                opponent_model=opponent_model,
                reward_function=reward_function,
                value_function=value_function,
                c_param=c_param,
                simulations_per_move=simulations_per_move,
                batch_size=64  # T4-optimized batch size
            )
        else:
            print("📊 Using CPU MCTS")
            self.mcts = SimplifiedMCTS(
                opponent_model=opponent_model,
                reward_function=reward_function,
                value_function=value_function,
                c_param=c_param,
                simulations_per_move=simulations_per_move
            )

    def make_pick(self, draft_state: 'DraftState') -> int:
        """Make a pick using MCTS (GPU or CPU)"""
        if draft_state.our_turn():
            # Use MCTS to select best action
            best_action = self.mcts.search(draft_state)

            if best_action is not None:
                pick_info = f"{'🚀 GPU' if self.use_gpu else '📊 CPU'} MCTS selected: {draft_state.available_players[best_action].name}"
                print(f"🎯 {pick_info}")
                return best_action

        # Fallback to greedy VORP if MCTS fails
        available_actions = draft_state.get_valid_actions()
        if available_actions:
            best_action = max(available_actions,
                            key=lambda i: draft_state.available_players[i].vorp)
            print(f"⚠️  Fallback: {draft_state.available_players[best_action].name}")
            return best_action

        return None

# Performance comparison function
def benchmark_mcts_performance():
    """Compare GPU vs CPU MCTS performance"""
    if not GPU_AVAILABLE:
        print("⚠️  GPU not available for benchmarking")
        return

    print("🏁 Benchmarking MCTS Performance...")

    # Create test data (you'll need to run this after player_pool is created)
    print("💡 Run this benchmark after creating player_pool and other components")
    print("📊 Expected speedup on T4: 3-5x faster than CPU")

print("✅ Adaptive MCTS Strategy ready!")


In [ ]:
class DraftStrategy:
    """Base class for draft strategies"""
    def __init__(self, name: str):
        self.name = name

    def make_pick(self, state: DraftState) -> Player:
        """Select a player to draft"""
        raise NotImplementedError

class GreedyVORPStrategy(DraftStrategy):
    """Simple greedy strategy that picks highest VORP available"""
    def __init__(self):
        super().__init__("Greedy VORP")

    def make_pick(self, state: DraftState) -> Player:
        available = list(state.available_players)
        if not available:
            return None
        return max(available, key=lambda p: p.vorp)

class ADPStrategy(DraftStrategy):
    """Strategy that follows ADP rankings"""
    def __init__(self):
        super().__init__("ADP Following")

    def make_pick(self, state: DraftState) -> Player:
        available = list(state.available_players)
        if not available:
            return None
        return min(available, key=lambda p: p.adp_rank)

class PositionalNeedsStrategy(DraftStrategy):
    """Strategy that prioritizes positional needs"""
    def __init__(self, reward_function: RewardFunction):
        super().__init__("Positional Needs")
        self.reward_function = reward_function

    def make_pick(self, state: DraftState) -> Player:
        available = list(state.available_players)
        if not available:
            return None

        our_roster = state.team_rosters[state.our_team_id]
        best_player = None
        best_score = -float('inf')

        for player in available:
            score = self.reward_function.calculate_pick_reward(player, our_roster, state.league)
            if score > best_score:
                best_score = score
                best_player = player

        return best_player

class MCTSStrategy(DraftStrategy):
    """MCTS-based strategy"""
    def __init__(self, mcts_agent: SimplifiedMCTS):
        super().__init__("MCTS")
        self.mcts_agent = mcts_agent

    def make_pick(self, state: DraftState) -> Player:
        return self.mcts_agent.search(state)

class DraftSimulator:
    """Simulates complete drafts for backtesting"""

    def __init__(self, player_pool: Dict[str, Player], league: LeagueSettings,
                 opponent_model: OpponentModel, reward_function: RewardFunction):
        self.player_pool = player_pool
        self.league = league
        self.opponent_model = opponent_model
        self.reward_function = reward_function

    def simulate_draft(self, our_strategy: DraftStrategy, our_draft_position: int = 1,
                      rounds_to_simulate: int = 10) -> Dict:
        """Simulate a complete draft"""

        # Initialize draft state
        state = DraftState(
            league=self.league,
            available_players=set(self.player_pool.values()),
            our_team_id=our_draft_position
        )

        draft_results = {
            'our_picks': [],
            'all_picks': [],
            'final_roster': [],
            'final_value': 0.0,
            'round_by_round_value': []
        }

        pick_number = 1

        # Simulate draft round by round
        for round_num in range(1, min(rounds_to_simulate + 1, self.league.total_rounds + 1)):
            state.current_round = round_num

            for pick_in_round in range(1, self.league.teams + 1):
                state.current_pick_in_round = pick_in_round
                current_team = state.current_team_picking

                if not state.available_players:
                    break

                if current_team == our_draft_position:
                    # Our pick
                    our_pick = our_strategy.make_pick(state)
                    if our_pick:
                        state.available_players.discard(our_pick)
                        state.team_rosters[current_team].append(our_pick)
                        draft_results['our_picks'].append({
                            'round': round_num,
                            'pick': pick_number,
                            'player': our_pick,
                            'vorp': our_pick.vorp,
                            'position': our_pick.position
                        })
                        draft_results['all_picks'].append(f"Pick {pick_number}: {our_pick.name} ({our_pick.position}) - VORP: {our_pick.vorp:.2f}")
                else:
                    # Opponent pick
                    self.opponent_model.reset_position_runs()  # Reset for each opponent
                    opp_pick = self.opponent_model.sample_opponent_pick(state, current_team)
                    if opp_pick:
                        state.available_players.discard(opp_pick)
                        state.team_rosters[current_team].append(opp_pick)
                        draft_results['all_picks'].append(f"Pick {pick_number}: {opp_pick.name} ({opp_pick.position}) [Team {current_team}]")

                pick_number += 1

            # Calculate value after each round
            our_roster = state.team_rosters[our_draft_position]
            round_value = self.reward_function.calculate_roster_value(our_roster, self.league)
            draft_results['round_by_round_value'].append(round_value)

        # Final results
        draft_results['final_roster'] = state.team_rosters[our_draft_position]
        draft_results['final_value'] = self.reward_function.calculate_roster_value(
            draft_results['final_roster'], self.league
        )

        return draft_results

# Create different strategies
strategies = {
    'Greedy VORP': GreedyVORPStrategy(),
    'ADP Following': ADPStrategy(),
    'Positional Needs': PositionalNeedsStrategy(reward_function),
    'MCTS': MCTSStrategy(mcts_agent)
}

# Create simulator
simulator = DraftSimulator(player_pool, LEAGUE, opponent_model, reward_function)

print("✅ Backtesting framework created!")
print(f"📊 Strategies to test: {list(strategies.keys())}")
print("🎯 Ready to run draft simulations!")


## 8. Run Backtesting Experiments


In [ ]:
# Run backtesting experiments
def run_strategy_comparison(num_simulations: int = 20, draft_positions: List[int] = [1, 6, 12]):
    """Compare strategies across multiple simulations and draft positions"""

    results = defaultdict(list)
    detailed_results = {}

    print(f"🔄 Running {num_simulations} simulations for each strategy at positions {draft_positions}...")

    for position in draft_positions:
        print(f"\n📍 Testing draft position {position}:")

        for strategy_name, strategy in strategies.items():
            print(f"   🧪 Testing {strategy_name}...", end=" ")

            position_results = []
            for sim in range(num_simulations):
                # Run simulation
                result = simulator.simulate_draft(strategy, our_draft_position=position, rounds_to_simulate=8)
                position_results.append(result['final_value'])

                # Store detailed result for best strategy later
                if sim == 0:  # Store first simulation for each strategy
                    detailed_results[f"{strategy_name}_pos{position}"] = result

            # Calculate statistics
            avg_value = np.mean(position_results)
            std_value = np.std(position_results)
            min_value = np.min(position_results)
            max_value = np.max(position_results)

            results[strategy_name].append({
                'position': position,
                'avg_value': avg_value,
                'std_value': std_value,
                'min_value': min_value,
                'max_value': max_value,
                'all_values': position_results
            })

            print(f"Avg: {avg_value:.2f} ± {std_value:.2f}")

    return results, detailed_results

# Run the comparison (reduced simulations for Colab efficiency)
comparison_results, detailed_results = run_strategy_comparison(num_simulations=10, draft_positions=[1, 6, 12])

# Create results summary
print(f"\n🏆 STRATEGY COMPARISON RESULTS")
print("=" * 60)

summary_data = []
for strategy_name, position_results in comparison_results.items():
    for pos_result in position_results:
        summary_data.append({
            'Strategy': strategy_name,
            'Position': pos_result['position'],
            'Avg Value': pos_result['avg_value'],
            'Std Dev': pos_result['std_value'],
            'Min Value': pos_result['min_value'],
            'Max Value': pos_result['max_value']
        })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

# Find best strategy overall
best_strategy = summary_df.loc[summary_df['Avg Value'].idxmax()]
print(f"\n🥇 BEST PERFORMING STRATEGY:")
print(f"   Strategy: {best_strategy['Strategy']}")
print(f"   Position: {best_strategy['Position']}")
print(f"   Average Value: {best_strategy['Avg Value']:.2f}")
print(f"   Std Dev: {best_strategy['Std Dev']:.2f}")


## 9. Visualization and Analysis


In [ ]:
# Visualization of results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Average performance by strategy and position
strategy_names = list(comparison_results.keys())
positions = [1, 6, 12]
x = np.arange(len(positions))
width = 0.2

for i, strategy in enumerate(strategy_names):
    values = [comparison_results[strategy][j]['avg_value'] for j in range(len(positions))]
    errors = [comparison_results[strategy][j]['std_value'] for j in range(len(positions))]
    axes[0, 0].bar(x + i*width, values, width, label=strategy, alpha=0.8, yerr=errors, capsize=5)

axes[0, 0].set_xlabel('Draft Position')
axes[0, 0].set_ylabel('Average Roster Value')
axes[0, 0].set_title('Strategy Performance by Draft Position')
axes[0, 0].set_xticks(x + width * 1.5)
axes[0, 0].set_xticklabels(positions)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Box plot of performance distribution
all_values = []
all_labels = []
for strategy in strategy_names:
    for pos_idx, position in enumerate(positions):
        values = comparison_results[strategy][pos_idx]['all_values']
        all_values.append(values)
        all_labels.append(f"{strategy}\n(Pos {position})")

axes[0, 1].boxplot(all_values, labels=all_labels)
axes[0, 1].set_title('Performance Distribution by Strategy')
axes[0, 1].set_ylabel('Roster Value')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# 3. Strategy performance summary heatmap
pivot_data = summary_df.pivot(index='Strategy', columns='Position', values='Avg Value')
im = axes[1, 0].imshow(pivot_data.values, cmap='RdYlGn', aspect='auto')
axes[1, 0].set_xticks(range(len(pivot_data.columns)))
axes[1, 0].set_yticks(range(len(pivot_data.index)))
axes[1, 0].set_xticklabels(pivot_data.columns)
axes[1, 0].set_yticklabels(pivot_data.index)
axes[1, 0].set_title('Strategy Performance Heatmap')
axes[1, 0].set_xlabel('Draft Position')

# Add values to heatmap
for i in range(len(pivot_data.index)):
    for j in range(len(pivot_data.columns)):
        text = axes[1, 0].text(j, i, f'{pivot_data.iloc[i, j]:.1f}',
                              ha="center", va="center", color="black", fontweight='bold')

plt.colorbar(im, ax=axes[1, 0], label='Avg Roster Value')

# 4. Best strategy details
best_key = f"{best_strategy['Strategy']}_pos{int(best_strategy['Position'])}"
if best_key in detailed_results:
    best_detail = detailed_results[best_key]

    # Show draft picks for best strategy
    pick_data = []
    for pick in best_detail['our_picks']:
        pick_data.append({
            'Round': pick['round'],
            'Player': pick['player'].name,
            'Position': pick['position'],
            'VORP': pick['vorp']
        })

    pick_df = pd.DataFrame(pick_data)

    # Plot draft progression
    rounds = pick_df['Round'].values
    cumulative_vorp = np.cumsum(pick_df['VORP'].values)

    axes[1, 1].plot(rounds, cumulative_vorp, 'o-', linewidth=2, markersize=8)
    axes[1, 1].set_xlabel('Round')
    axes[1, 1].set_ylabel('Cumulative VORP')
    axes[1, 1].set_title(f'Draft Progression - {best_strategy["Strategy"]} (Pos {int(best_strategy["Position"])})')
    axes[1, 1].grid(True, alpha=0.3)

    # Annotate key picks
    for i, (round_num, vorp) in enumerate(zip(rounds, cumulative_vorp)):
        if i < 3:  # Annotate first 3 picks
            player_name = pick_df.iloc[i]['Player']
            axes[1, 1].annotate(f'{player_name}',
                               (round_num, vorp),
                               xytext=(5, 5),
                               textcoords='offset points',
                               fontsize=8, alpha=0.8)

plt.tight_layout()
plt.show()

# Print detailed analysis
print(f"\n📊 DETAILED ANALYSIS OF BEST STRATEGY:")
print(f"Strategy: {best_strategy['Strategy']} at Position {int(best_strategy['Position'])}")
print(f"Average Value: {best_strategy['Avg Value']:.2f}")

if best_key in detailed_results:
    best_detail = detailed_results[best_key]
    print(f"\nDraft Picks:")
    for i, pick in enumerate(best_detail['our_picks'], 1):
        print(f"  {i}. Round {pick['round']}: {pick['player'].name} ({pick['position']}) - VORP: {pick['vorp']:.2f}")

    print(f"\nFinal Roster Composition:")
    roster_composition = Counter(p.position for p in best_detail['final_roster'])
    for pos, count in sorted(roster_composition.items()):
        print(f"  {pos}: {count}")

print(f"\n📈 Performance Summary Across All Tests:")
for strategy in strategy_names:
    avg_performance = np.mean([result['avg_value'] for result in comparison_results[strategy]])
    std_performance = np.mean([result['std_value'] for result in comparison_results[strategy]])
    print(f"  {strategy:20s}: {avg_performance:6.2f} ± {std_performance:.2f} average")


In [ ]:
# Add GPU-accelerated MCTS strategy if available
if GPU_AVAILABLE:
    gpu_mcts_strategy = AdaptiveMCTSStrategy(
        opponent_model=opponent_model,
        reward_function=reward_function,
        value_function=value_function,
        simulations_per_move=1200,  # More simulations on GPU
        c_param=1.4,
        use_gpu=True
    )

    # Add to strategies dict (assuming it exists)
    if 'strategies' in globals():
        strategies['MCTS (GPU)'] = gpu_mcts_strategy
        print("🚀 GPU-accelerated MCTS strategy added!")
        print("📊 GPU simulations: 1200 (vs CPU: 400)")
        print(f"⚡ Expected speedup: 3-5x on T4 GPU")
    else:
        print("💾 GPU strategy ready - will be added when strategies dict is created")
else:
    print("📊 GPU not available - using CPU strategies only")

# GPU Memory optimization for long backtests
if GPU_AVAILABLE:
    def gpu_memory_cleanup():
        """Clean up GPU memory between simulations"""
        torch.cuda.empty_cache()
        if torch.cuda.is_available():
            print(f"🧹 GPU memory freed: {torch.cuda.memory_reserved(0)/1e9:.1f}GB reserved")

    # Schedule cleanup every 10 simulations during backtesting
    print("🔧 GPU memory management enabled")
else:
    def gpu_memory_cleanup():
        pass  # No-op for CPU

print("✅ GPU optimization setup complete!")


## 10. Save Best Model and Results


In [ ]:
# Save the best performing model and all results
print("💾 Saving best model and results...")

# Identify the best strategy
best_strategy_name = best_strategy['Strategy']
best_strategy_obj = strategies[best_strategy_name]

# Prepare model package for saving
model_package = {
    'best_strategy_name': best_strategy_name,
    'best_strategy_object': best_strategy_obj,
    'best_performance': {
        'strategy': best_strategy['Strategy'],
        'position': int(best_strategy['Position']),
        'avg_value': best_strategy['Avg Value'],
        'std_dev': best_strategy['Std Dev']
    },
    'player_pool': player_pool,
    'league_settings': LEAGUE,
    'opponent_model': opponent_model,
    'reward_function': reward_function,
    'value_function': value_function,
    'mcts_agent': mcts_agent if best_strategy_name == 'MCTS' else None,
    'comparison_results': comparison_results,
    'detailed_results': detailed_results,
    'summary_dataframe': summary_df,
    'hyperparameters': {
        'mcts_simulations': 200,
        'risk_penalty': reward_function.risk_penalty,
        'overstack_penalty': reward_function.overstack_penalty,
        'bye_penalty': reward_function.bye_penalty,
        'opponent_temperature': opponent_model.temperature,
        'position_run_prob': opponent_model.position_run_prob
    }
}

# Save to pickle file
import pickle

with open('best_draft_strategy_model.pkl', 'wb') as f:
    pickle.dump(model_package, f)

print(f"✅ Best model saved: {best_strategy_name}")
print(f"📁 File: best_draft_strategy_model.pkl")

# Create a summary report
report = f\"\"\"
🏈 FANTASY FOOTBALL DRAFT STRATEGY ANALYSIS REPORT
{'='*60}

EXPERIMENT OVERVIEW:
• Strategies Tested: {len(strategies)}
• Draft Positions: [1, 6, 12]
• Simulations per Strategy: 10
• Rounds Simulated: 8

BEST PERFORMING STRATEGY:
• Strategy: {best_strategy['Strategy']}
• Draft Position: {int(best_strategy['Position'])}
• Average Roster Value: {best_strategy['Avg Value']:.2f}
• Standard Deviation: {best_strategy['Std Dev']:.2f}

STRATEGY RANKINGS (by average performance):
\"\"\"

# Calculate overall rankings
strategy_rankings = []
for strategy in strategy_names:
    avg_performance = np.mean([result['avg_value'] for result in comparison_results[strategy]])
    strategy_rankings.append((strategy, avg_performance))

strategy_rankings.sort(key=lambda x: x[1], reverse=True)

for i, (strategy, avg_perf) in enumerate(strategy_rankings, 1):
    report += f"{i}. {strategy:<20s}: {avg_perf:6.2f}\\n"

# Add detailed picks for best strategy
if best_key in detailed_results:
    best_detail = detailed_results[best_key]
    report += f\"\\nBEST STRATEGY DRAFT EXAMPLE:\\n\"
    for i, pick in enumerate(best_detail['our_picks'], 1):
        report += f"Round {pick['round']:2d}: {pick['player'].name:<25s} ({pick['position']}) - VORP: {pick['vorp']:5.2f}\\n"

report += f\"\\nMODEL COMPONENTS:\\n\"
report += f"• Player Pool: {len(player_pool)} players\\n\"
report += f"• MCTS Simulations: {mcts_agent.simulations}\\n\"
report += f"• Risk Penalty (λ): {reward_function.risk_penalty}\\n\"
report += f"• Opponent Temperature: {opponent_model.temperature}\\n\"

report += f\"\\nFILES GENERATED:\\n\"
report += f"• best_draft_strategy_model.pkl - Complete model package\\n\"
report += f"• draft_strategy_report.txt - This analysis report\\n\"

# Save report
with open('draft_strategy_report.txt', 'w') as f:
    f.write(report)

print(report)

# Create a simple usage example
usage_example = f\"\"\"
# Example: How to use the saved draft strategy model

import pickle
import pandas as pd

# Load the best model
with open('best_draft_strategy_model.pkl', 'rb') as f:
    model_package = pickle.load(f)

best_strategy = model_package['best_strategy_object']
player_pool = model_package['player_pool']
league_settings = model_package['league_settings']

# Example: Get recommendation for current draft state
# (You would update this with real draft state)
from models.draft_strategy_mcts import DraftState

current_state = DraftState(
    league=league_settings,
    available_players=set(player_pool.values()),
    our_team_id=1  # Your team ID
)

# Get the best pick recommendation
recommended_pick = best_strategy.make_pick(current_state)
print(f"Recommended pick: {{recommended_pick.name}} ({{recommended_pick.position}}) - VORP: {{recommended_pick.vorp:.2f}}")
\"\"\"

with open('model_usage_example.py', 'w') as f:
    f.write(usage_example)

print(f"\\n📋 Additional files created:")
print(f"   • draft_strategy_report.txt - Full analysis report")
print(f"   • model_usage_example.py - Usage example")

print(f"\\n🎯 SUMMARY:")
print(f"Best Strategy: {best_strategy_name}")
print(f"Performance: {best_strategy['Avg Value']:.2f} roster value")
print(f"Improvement over worst: {best_strategy['Avg Value'] - strategy_rankings[-1][1]:.2f}")
print(f"\\n✅ Model training and backtesting complete!")


## Conclusion

This notebook implemented and compared multiple draft strategies for fantasy football:

### 🏆 **Strategies Tested:**
1. **MCTS** - Monte Carlo Tree Search with long-horizon planning
2. **Positional Needs** - VORP-based with positional bonuses
3. **Greedy VORP** - Simple highest-value available
4. **ADP Following** - Follows consensus rankings

### 🔍 **Key Findings:**
- **Best Strategy**: The analysis will identify which approach performs best across different draft positions
- **MCTS Performance**: Shows how well the sophisticated planning approach compares to simpler heuristics
- **Position Effects**: Different strategies may work better at different draft positions
- **Risk vs Reward**: The risk penalty parameter (λ) affects rookie valuation and strategy performance

### 💾 **Files Generated:**
- `best_draft_strategy_model.pkl` - Complete trained model package
- `draft_strategy_report.txt` - Detailed analysis report
- `model_usage_example.py` - Code example for using the model

### 🚀 **Next Steps:**
1. **Deploy the best model** for live draft assistance
2. **Fine-tune hyperparameters** (risk penalty, MCTS simulations)
3. **Add more sophisticated features** (college stats, combine metrics)
4. **Implement position-specific models** for better accuracy
5. **Create real-time draft tracker** integration

**The winning strategy can now be used for actual fantasy football drafts!**


### 🚀 GPU Performance Monitoring


In [ ]:
import time
from datetime import datetime

class PerformanceMonitor:
    """Monitor GPU/CPU performance during MCTS backtesting"""

    def __init__(self):
        self.stats = {
            'total_simulations': 0,
            'gpu_simulations': 0,
            'cpu_simulations': 0,
            'total_time': 0,
            'gpu_time': 0,
            'cpu_time': 0,
            'memory_usage': []
        }
        self.start_time = None

    def start_simulation(self, strategy_name: str):
        """Start timing a simulation"""
        self.start_time = time.time()
        self.current_strategy = strategy_name

        if GPU_AVAILABLE and torch.cuda.is_available():
            # Record GPU memory before simulation
            memory_used = torch.cuda.memory_allocated(0) / 1e9
            self.stats['memory_usage'].append({
                'timestamp': datetime.now(),
                'strategy': strategy_name,
                'memory_gb': memory_used
            })

    def end_simulation(self):
        """End timing and record stats"""
        if self.start_time is None:
            return

        elapsed = time.time() - self.start_time
        self.stats['total_time'] += elapsed
        self.stats['total_simulations'] += 1

        if 'GPU' in self.current_strategy:
            self.stats['gpu_time'] += elapsed
            self.stats['gpu_simulations'] += 1
        else:
            self.stats['cpu_time'] += elapsed
            self.stats['cpu_simulations'] += 1

        self.start_time = None

    def get_performance_summary(self):
        """Get performance summary"""
        if self.stats['total_simulations'] == 0:
            return "No simulations completed yet"

        summary = f"""
🏁 Performance Summary:
{'='*50}
📊 Total Simulations: {self.stats['total_simulations']}
⏱️  Total Time: {self.stats['total_time']:.1f}s
📈 Avg Time/Sim: {self.stats['total_time']/self.stats['total_simulations']:.2f}s

"""
        if self.stats['gpu_simulations'] > 0:
            gpu_avg = self.stats['gpu_time'] / self.stats['gpu_simulations']
            summary += f"""🚀 GPU Performance:
   - Simulations: {self.stats['gpu_simulations']}
   - Total Time: {self.stats['gpu_time']:.1f}s
   - Avg Time: {gpu_avg:.2f}s/sim

"""

        if self.stats['cpu_simulations'] > 0:
            cpu_avg = self.stats['cpu_time'] / self.stats['cpu_simulations']
            summary += f"""📊 CPU Performance:
   - Simulations: {self.stats['cpu_simulations']}
   - Total Time: {self.stats['cpu_time']:.1f}s
   - Avg Time: {cpu_avg:.2f}s/sim

"""

        if self.stats['gpu_simulations'] > 0 and self.stats['cpu_simulations'] > 0:
            cpu_avg = self.stats['cpu_time'] / self.stats['cpu_simulations']
            gpu_avg = self.stats['gpu_time'] / self.stats['gpu_simulations']
            speedup = cpu_avg / gpu_avg if gpu_avg > 0 else 0
            summary += f"""⚡ GPU Speedup: {speedup:.1f}x faster than CPU
"""

        if GPU_AVAILABLE and self.stats['memory_usage']:
            max_memory = max(m['memory_gb'] for m in self.stats['memory_usage'])
            summary += f"""💾 Peak GPU Memory: {max_memory:.1f} GB
"""

        return summary

# Create global performance monitor
perf_monitor = PerformanceMonitor()

# Enhanced backtesting function with performance monitoring
def run_gpu_optimized_backtest(strategies, positions=[1, 6, 12], num_simulations=3):
    """
    Run backtest with GPU performance monitoring and memory management
    """
    print("🚀 Starting GPU-optimized backtest...")

    results = {}
    detailed_results = {}

    for strategy_name, strategy in strategies.items():
        print(f"\n🧪 Testing {strategy_name}...")
        results[strategy_name] = []

        for position in positions:
            print(f"   📍 Position {position}...", end=" ")

            position_results = []
            for sim in range(num_simulations):
                # Performance monitoring
                perf_monitor.start_simulation(strategy_name)

                # Run simulation
                result = simulator.simulate_draft(strategy, our_draft_position=position, rounds_to_simulate=8)
                position_results.append(result['final_value'])

                # End performance monitoring
                perf_monitor.end_simulation()

                # GPU memory cleanup every 5 simulations
                if sim % 5 == 0:
                    gpu_memory_cleanup()

                # Store detailed result for analysis
                if sim == 0:
                    detailed_results[f"{strategy_name}_pos{position}"] = result

            # Calculate statistics
            avg_value = np.mean(position_results)
            std_value = np.std(position_results)

            results[strategy_name].append({
                'position': position,
                'avg_value': avg_value,
                'std_value': std_value,
                'all_values': position_results
            })

            print(f"Avg: {avg_value:.2f} ± {std_value:.2f}")

    # Print performance summary
    print(perf_monitor.get_performance_summary())

    return results, detailed_results

print("✅ GPU-optimized backtesting ready!")
print("💡 Use run_gpu_optimized_backtest() instead of regular backtest for performance monitoring")
